<a href="https://colab.research.google.com/github/KU-ZHANGZIYUE/RainfallForecasting_ConvEncDec/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Convolutional Encoder Decoder Model
#Author: ZHANG Ziyue (Kyoto University)
#Date: 2011/11/3

from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Research/5mins_4ch

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Research/5mins_4ch


In [5]:
import numpy as np
import os
from keras.models import *
from keras.layers import *
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers


In [ ]:
def test():
    
    inputs = Input((192, 128, 4))

    conv1f = Conv2D(64, 3, padding='same')(inputs)
    conv1f = BatchNormalization()(conv1f)
    conv1f = Activation("relu")(conv1f)
    conv1f = Conv2D(64, 3, padding='same')(conv1f)
    conv1f = BatchNormalization()(conv1f)
    conv1f = Activation("relu")(conv1f)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1f)

    conv2f = Conv2D(128, 3, padding='same')(pool1)
    conv2f = BatchNormalization()(conv2f)
    conv2f = Activation("relu")(conv2f)
    conv2f = Conv2D(128, 3, padding='same')(conv2f)
    conv2f = BatchNormalization()(conv2f)
    conv2f = Activation("relu")(conv2f)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2f)

    conv3f = Conv2D(256, 3, padding='same')(pool2)
    conv3f = BatchNormalization()(conv3f)
    conv3f = Activation("relu")(conv3f)
    conv3f = Conv2D(256, 3, padding='same')(conv3f)
    conv3f = BatchNormalization()(conv3f)
    conv3f = Activation("relu")(conv3f)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3f)

    conv4f = Conv2D(256, 3, padding='same')(pool3)
    conv4f = BatchNormalization()(conv4f)
    conv4f = Activation("relu")(conv4f)
    conv4f = Conv2D(256, 3, padding='same')(conv4f)
    conv4f = BatchNormalization()(conv4f)
    conv4f = Activation("relu")(conv4f)
    up1 = UpSampling2D((2, 2))(conv4f)

    conv5f = Conv2D(128, 3, padding='same')(up1)
    conv5f = BatchNormalization()(conv5f)
    conv5f = Activation("relu")(conv5f)
    conv5f = Conv2D(128, 3, padding='same')(conv5f)
    conv5f = BatchNormalization()(conv5f)
    conv5f = Activation("relu")(conv5f)
    up2 = UpSampling2D((2, 2))(conv5f)

    conv6f = Conv2D(64, 3, padding='same')(up2)
    conv6f = BatchNormalization()(conv6f)
    conv6f = Activation("relu")(conv6f)
    conv6f = Conv2D(64, 3, padding='same')(conv6f)
    conv6f = BatchNormalization()(conv6f)
    conv6f = Activation("relu")(conv6f)
    up3 = UpSampling2D((2, 2))(conv6f)

    outputs = Conv2D(1, 1, activation='linear')(up3)

    model = Model(inputs=inputs, outputs=outputs)

    return model
print(test().summary())

#然后validation改成20%，这个意思就是这最后20不会被用来traintrain
#然后挑选"validation"也就是最后20%的data中的几个sample，plot出来input，prediction和ground truth

In [ ]:
input = np.load('./paddedinput_5mins_4ch.npy')
output = np.load('./paddedoutput_5mins_4ch.npy')

model = test()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='mean_squared_error', metrics=['accuracy'])
result = model.fit(input, output, batch_size=8, epochs=30, validation_split=0.2) #val may be 20%
model.save('result_initial_5min_4ch.h5')

In [ ]:
plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
#plt.ylim(-0.1, 0.2)
plt.show()

In [ ]:
plt.plot(result.history['accuracy'])
plt.plot(result.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()